In [7]:
import sys
from pathlib import Path
from typing import Iterable

from netlistio.ingestor.compiler import Compiler
from netlistio.ingestor.linker import link
from netlistio.ingestor.parser import Parser
from netlistio.ingestor.registry import ModelRegistry
from netlistio.ingestor.scanner import Scanner
from netlistio.ingestor.spice import SpiceChunkParserFactory, SpiceScanStrategy
from netlistio.models.generic import Primitive
from netlistio.models.parsing import ParseRegion
from netlistio.models.spice import SpiceNetlist, prefix_registry
from netlistio.graph_analysis.circuit_graph import CircuitGraph


def _scanner_factory(filepath: Path) -> Scanner:
    """Creates a Scanner configured for SPICE."""
    return Scanner(filepath, SpiceScanStrategy())


def _parser_factory(filepath: Path, regions: Iterable[ParseRegion]) -> Parser:
    """Creates a Parser configured for SPICE."""
    return Parser(filepath, regions, SpiceChunkParserFactory())

In [8]:
netlist_path = "/workspaces/NetlistIO/tests/data/foundry/sram/Prelayout/Spice_models/sense_amplifier.spice"
root_file = Path(netlist_path)
compiler = Compiler(
    root_filepath=root_file,
    parser_factory=_parser_factory,
    scanner_factory=_scanner_factory,
)
parse_result = compiler.compile()
print(f"Compilation Complete.")
print(f"  Files processed: {len(compiler.visited_regions)}")
print(f"  Cells parsed: {len(parse_result.cells)}")
print(f"  Errors: {len(parse_result.errors)}")
for error in parse_result.errors:
    print(f"    [Parse Error] {error.message} (Line {error.line_number})")

Compilation Complete.
  Files processed: 10
  Cells parsed: 9
  Errors: 0


In [9]:
model_registry = ModelRegistry(
    static_primitives={
        primitive.name.lower(): primitive
        for primitive in prefix_registry().values()
        if isinstance(primitive, Primitive)
    }
)

In [10]:
print("\n--- Linking ---")
link_result = link(parse_result, model_registry, SpiceNetlist)
netlist = link_result.netlist

print(f"Linking Complete.")
print(f"  Top-level instances: {len(link_result.top_instances)}")
print(f"  Link Errors: {len(link_result.errors)}")

for error in link_result.errors:
    print(f"    [Link Error] {error.message}: {', '.join(error.affected_cells)}")


--- Linking ---
Linking Complete.
  Top-level instances: 9
  Link Errors: 0


In [11]:
print("\n--- Netlist Dump ---")
for cell in netlist.cells:
    cell.write(sys.stdout)


--- Netlist Dump ---
PMOS: pmos
    Port: d
    Port: g
    Port: s
    Port: b
Subckt: sky130_fd_pr__pfet_01v8
    Instance: msky130_fd_pr__pfet_01v8
        Port: d -> Net: d
        Port: g -> Net: g
        Port: s -> Net: s
        Port: b -> Net: b
        nf={nf}, sd={sd}, sb={sb}, sa={sa}, nrs={nrs}, nrd={nrd}, ps={ps}, pd={pd}, as={as}, ad={ad}, w={w}, l={l}
        Model:
            PMOS: pmos
                Port: d
                Port: g
                Port: s
                Port: b
Subckt: sky130_fd_pr__nfet_01v8
    Instance: msky130_fd_pr__nfet_01v8
        Port: d -> Net: d
        Port: g -> Net: g
        Port: s -> Net: s
        Port: b -> Net: b
        nf={nf}, sd={sd}, sb={sb}, sa={sa}, nrs={nrs}, nrd={nrd}, ps={ps}, pd={pd}, as={as}, ad={ad}, w={w}, l={l}
        Model:
            PMOS: pmos
                Port: d
                Port: g
                Port: s
                Port: b
Instance: XM1
    Port: d -> Net: net1
    Port: g -> Net: blb
    Port

In [12]:
cg = CircuitGraph.from_netlist(netlist)